In [ ]:
import plotly.graph_objects as go
from scipy.stats import norm
import math
import numpy as np

# Valuation

## Estimation of the cost of equity

## How to de-/re-leverage a beta

## Derivation of delevered (or unlevered) beta:

## WACC

In [1]:
# CAPM
def calculate_re(rf, beta, rm):
    '''
    Calculates re, the cost of an equity
    Parameters
    ----------
    rf: float
    risk-free rate
    beta: float
    beta of the stock (market risk)
    rm: float
    market return
    Notes
    -----
    rm-rf is the premium of the market over the risk-free rate
    '''
    re = rf + beta*(rm-rf)
    return re

# Hamada-Equation, supposing beta_debt=0
def hamada(beta_u, Vd, Ve, tau):
    return beta_u*(1 + Vd/Ve*(1-tau))

def hamada_(beta_e, Vd, Ve, tau):
    return beta_e*(1/(1 + Vd/Ve*(1-tau)))
    
def calculate_wacc(ke, kd, Ve, Vd, tau):
    V = Ve + Vd
    return ke*Ve/V + kd*(1-tau)*Vd/V

In [2]:
hamada(.8, 9, 6, .3)

1.64

In [3]:
Vd_comp = 2
Ve_comp = 71
tau_comp = .33
beta_e_comp = 1.24
beta_u_comp = hamada_(beta_e_comp, Vd_comp, Ve_comp, tau_comp)
print(f"{beta_u_comp=:.2f}")
# We assume beta_u_comp = beta_u
beta_u = beta_u_comp
Vd = 4
Ve = 11.5
tau = .33
beta = hamada(beta_u, Vd, Ve, tau)
print(f"{beta=:.2f}")
# cost of equity
rf = .076
rm = .121
re = calculate_re(rf, beta, rm)
print(f"{re=:.2f}")
# WACC
ke = re
kd = .1
Ve = 1150
Vd = 400
WACC = calculate_wacc(ke, kd, Ve, Vd, tau)
print(f"{WACC=:.2f}")

beta_u_comp=1.22
beta=1.50
re=0.14
WACC=0.12


## Present Value (PV)

In [4]:
def calculate_PV(cashflows, rate):
    '''
    Calculates the Present Value of a company based on the projection of the cashflo
    cashflows: list[float]
    cashflows
    rate: float
    constant rate utilized, can be WACC
    '''
    t = len(cashflows)
    PV = 0
    for i in range(t):
        PV += cashflows[i]/((1 + rate)**(i + 1))
    return PV

In [5]:
cashflows = [30, 35, 40, 45]
rate = 0.0756 # from last code-cell
PV = calculate_PV(cashflows, rate)
print(f"{PV=:.2f}")

PV=123.91


In [7]:
def get_entity_value_dcf(cashflows, rate, g):
    # we approximate the cashflow in t+1 with the value at t with growth g and rate
    terminal_value = cashflows[-1]*(1+g)/(rate-g)
    # get it to present value
    PV_final = terminal_value/(1 + rate)**(len(cashflows))
    PV = calculate_PV(cashflows, rate)
    V = PV + PV_final
    return V

In [8]:
cashflows = [30, 35, 40, 45]
g = .02
re = .1
rd = .05
Vd = 3
Ve = 5
WACC = calculate_wacc(tau, re, rd, Ve, Vd)
rate = WACC
V = get_entity_value_dcf(cashflows, rate, g)
print(f"{V=:.2f}")
print(f"Ve={V-Vd:.2f}")

V=-233.48
Ve=-236.48


## Specific AOV cases

In [10]:
def calculate_apv(re, rd, t, rf, rm, k, Vd=None, Ve=None):
    '''
    k=0: case I (constant leverage ratio) => rt = ru
    k=1: case II (constant debt level) =>
    rt = rd
    '''
    if Vd:
        Ve = 1 - Vd
    else:
        Vd = 1 - Ve
        
    ru = rd*Vd + re*Ve
    print(f"{ru=:.2f}")
    WACC = ru - Vd*t*(rd + k*(ru-rd))
    print(f"{WACC=:.2f}")
    betau = (ru - rf)/rm
    print(f"{betau=:.2f}")
    betae = (re - rf)/rm
    print(f"{betae=:.2f}")
    betad = (betau - Ve*betae)/Vd
    print(f"{betad=:.2f}")

In [11]:
re = 0.1
rd = 0.06
t = 0.35
rf = 0.02
rm = 0.06 # Maturity risk premium
k = 1 # case
calculate_apv(re, rd, t, rf, rm, k, Vd=.5)

ru=0.08
WACC=0.07
betau=1.00
betae=1.33
betad=0.67


## Multiple Valuation

In [13]:
####
p = 44.12
S = 247e6
#R = 129e9
#E = 9.27e9
B = 1441.4e6
EBIT = 187.6e6
#EBITDA = 13e9
#P_R = p*S/R
#P_E = p*S/E
#P_B = p*S/B
EV_EBIT = (p*S + B)/EBIT
#EV_EBITDA = (p*S + B)/EBITDA
#print(f"{P_R=:,.2f}")
#print(f"{P_E=:,.2f}")
#print(f"{P_B=:,.2f}")
print(f"{EV_EBIT=:,.2f}")
#print(f"{EV_EBITDA=:,.2f}")
####
(EV_EBIT*89.64e6 - 544.2e6)/83e6

EV_EBIT=65.77


64.47835856860277

In [15]:
def calculate_wacc(tau, re, rd, Ve, Vd):
    V = Ve + Vd
    WACC = (1 - tau)*rd*Vd/V + re*Ve/V
    return WACC

In [16]:
tau = .3
re = .1
rd = .05
Vd = 3
Ve = 5
calculate_wacc(tau, re, rd, Ve, Vd)

0.075625